In [1]:
import tensorflow as tf

In [2]:
from tensorflow import keras

In [3]:
import numpy as np

In [4]:
shakespeare_url = "https://homl.info/shakespeare"

In [5]:
filepath = keras.utils.get_file('shakespeare.txt',shakespeare_url)

In [6]:
with open(filepath) as f:
    shakespeare_text = f.read()

In [7]:
open(filepath)

<_io.TextIOWrapper name='C:\\Users\\Besitzer\\.keras\\datasets\\shakespeare.txt' mode='r' encoding='cp1252'>

In [8]:
filepath.count

<function str.count>

In [9]:
shakespeare_text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [10]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

In [11]:
tokenizer

In [12]:
#help(tokenizer)

In [13]:
tokenizer.fit_on_texts([shakespeare_text])

In [14]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [15]:
tokenizer.texts_to_sequences(['Vijay'])

[[26, 6, 33, 5, 16]]

In [16]:
max_id = len(tokenizer.word_index)

In [17]:
max_id

39

In [18]:
#tokenizer.word_index

In [19]:
#help(tokenizer.word_docs)

In [20]:
dataset_size = tokenizer.document_count

In [21]:
dataset_size

1

In [22]:
len(shakespeare_text)

1115394

In [23]:
dataset_size = len(shakespeare_text)

In [24]:
dataset_size

1115394

In [25]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [26]:
encoded.dtype

dtype('int32')

In [27]:
encoded.shape

(1115394,)

In [28]:
train_size = dataset_size * 5 // 100

In [29]:
train_size

55769

In [30]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [31]:
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [32]:
help(tf.data.Dataset.from_tensor_slices)

Help on function from_tensor_slices in module tensorflow.python.data.ops.dataset_ops:

from_tensor_slices(tensors)
    Creates a `Dataset` whose elements are slices of the given tensors.
    
    The given tensors are sliced along their first dimension. This operation
    preserves the structure of the input tensors, removing the first dimension
    of each tensor and using it as the dataset dimension. All input tensors
    must have the same size in their first dimensions.
    
    >>> # Slicing a 1D tensor produces scalar tensor elements.
    >>> dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
    >>> list(dataset.as_numpy_iterator())
    [1, 2, 3]
    
    >>> # Slicing a 2D tensor produces 1D tensor elements.
    >>> dataset = tf.data.Dataset.from_tensor_slices([[1, 2], [3, 4]])
    >>> list(dataset.as_numpy_iterator())
    [array([1, 2], dtype=int32), array([3, 4], dtype=int32)]
    
    >>> # Slicing a tuple of 1D tensors produces tuple elements containing
    >>> # scala

In [33]:
encoded[:50]

array([19,  5,  8,  7,  2,  0, 18,  5,  2,  5, 35,  1,  9, 23, 10, 21,  1,
       19,  3,  8,  1,  0, 16,  1,  0, 22,  8,  3, 18,  1,  1, 12,  0,  4,
        9, 15,  0, 19, 13,  8,  2,  6,  1,  8, 17,  0,  6,  1,  4,  8])

In [34]:
#list(dataset.as_numpy_iterator())

In [35]:
n_steps = 100

In [36]:
window_length = n_steps + 1

In [37]:
dataset = dataset.window(window_length,drop_remainder=True,shift=1)

In [38]:
dataset = dataset.flat_map(lambda window : window.batch(window_length))

In [39]:
batch_size = 32

In [40]:
dataset = dataset.shuffle(10000).batch(batch_size)

In [41]:
dataset = dataset.map(lambda windows : (windows[:,:-1],windows[:,1:]))

In [42]:
dataset = dataset.map(lambda X_batch, Y_batch : (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [43]:
dataset = dataset.prefetch(1)

In [44]:
model = keras.models.Sequential([
    keras.layers.GRU(128,return_sequences=True,input_shape=[None,max_id],dropout=0.2,recurrent_dropout=0.2),
    keras.layers.GRU(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy",optimizer="adam")


In [45]:
history = model.fit(dataset,epochs=1)

1740/1740 [==============================] - 393s 226ms/step - loss: 1.9561


In [46]:
history = model.fit(dataset,epochs=10)

Epoch 1/10
1740/1740 [==============================] - 413s 237ms/step - loss: 1.5582
Epoch 2/10
1740/1740 [==============================] - 415s 238ms/step - loss: 1.4564
Epoch 3/10
1740/1740 [==============================] - 401s 230ms/step - loss: 1.4029
Epoch 4/10
1740/1740 [==============================] - 403s 232ms/step - loss: 1.3661
Epoch 5/10
1740/1740 [==============================] - 415s 239ms/step - loss: 1.3393
Epoch 6/10
1740/1740 [==============================] - 418s 240ms/step - loss: 1.3188
Epoch 7/10
1740/1740 [==============================] - 422s 243ms/step - loss: 1.3011
Epoch 8/10
1740/1740 [==============================] - 419s 241ms/step - loss: 1.2868
Epoch 9/10
1740/1740 [==============================] - 482s 277ms/step - loss: 1.2735
Epoch 10/10
1740/1740 [==============================] - 697s 401ms/step - loss: 1.2625


In [47]:
history = model.fit(dataset,epochs=3)

Epoch 1/3
1740/1740 [==============================] - 686s 395ms/step - loss: 1.2530
Epoch 2/3
1740/1740 [==============================] - 705s 405ms/step - loss: 1.2439
Epoch 3/3
1740/1740 [==============================] - 688s 395ms/step - loss: 1.2360


In [48]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1 
    return tf.one_hot(X,max_id)

In [49]:
X_new = preprocess(['How are yo'])

In [50]:
X_new

<tf.Tensor: shape=(1, 10, 39), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.

In [51]:
Y_pred = model.predict_classes(X_new)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [52]:
Y_pred

array([[ 1, 13,  0,  6,  8,  1,  0,  7,  3, 13]], dtype=int64)

In [53]:
tokenizer.sequences_to_texts(Y_pred+1)[0][-1]

'u'

### Lets us generate some fake Shakespearean data

In [54]:
def next_char(text,temperature):
    X_new = preprocess([text])
    y_prob = model.predict(X_new)[0,-1:,:]
    rescaled_logits = tf.math.log(y_prob)/temperature
    char_id = tf.random.categorical(rescaled_logits,num_samples=1)+1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [55]:
def complete_text(text,n_chars=500,temperature=1):
    for _ in range(n_chars):
        text += next_char(text,temperature)
    return text

In [56]:
print(complete_text("a",temperature=0.2))

at the people,
when they have made them the common party,
when we have been such a perpetual consul.

menenius:
the marketplace, some brown, some blows
most did not to be stand for the blood who have been so many grave been seven them to be
monstrous marcius coriolanus come off,
and the blood upon the marketplace,
our world grave for the people,
our proud to be say him stand upon the common party,
which he should be bless to him as the common party,
when he had rather have not to be swords to do,


In [57]:
print(complete_text("p", temperature=0.2))

pine, and i think if he did
re-quicken'd them as the people,
our proud to be say him stand for the people,
our noble call'd af the people,
but the man i' the common many-headed and their best to do; for the world
be swords to the common many grave not to be
many grave not to see him stand for the common battle call'd
to see him stand for the people,
our world: i do owe the world: i thank the people,
when he will be blood to be some bloody for the people,
but the man i' the people,
who have see hi


In [58]:
print(complete_text("hello", temperature=0.2))

hellown the people,
our proud to be ingrateful power to the common made all the lives
and the man i' the people,
our proud to be some blows he had rather have he did
report them the common made all joy and to be
many stand for the blood of the people,
our proved with shunn coriolanus and to be some bloody for the people.

second citizen:
we are so many grave men the common many-headed and the common for the people,
when he had rather have the people,
when he had rather have the people,
i will be blow


In [59]:
model.save('shakespeare1.h5')

In [60]:
from tensorflow.keras.models import load_model

In [61]:
print(complete_text("hello", temperature=1))

hellos,
as your world: doing the world: marcius coriolanus
much mamaius i' the sounds and loves taken to the city,
let caius marcius brutus.

menenius:
she do speak to make a human praised was one cannot
be cannot to have him for hearing them. he hath we have
fast true! iw his bread and i shall
be nottemn cafath'd the people, let's field.

sicinius:
the fliers!
o the noble deeds; never thought
with the nicer, she have donation'd
way o'er the linds one this stay it's intening
with speak, some out of h


In [62]:
print(complete_text("p", temperature=1))

pent in all the blind. how is he cames the sun
with thind unoort: o that the miles.

menenius:
i had rather shall be sarr in blood
upon 't: but my word: and fair, he cannot act
that for noon, he stopp'd against the bleade had rather
gracious so our
the city, i fought the common for and report him good
mution.

brutus:
havill'd inclined in what come.

coriolanus:
you give us.

menenius:
i sin the people,
your fliers; neither welcome to the least call'd me, i that which
i will be the field when i t
